#### MVNcall - preparing data

In [1]:
import malariagen_data
import numpy as np
import pandas as pd
import allel
from datetime import date

import sys
# adding Folder_2 to the system path
sys.path.insert(0, '/home/sanj/projects/gaardian/workflow/scripts/')
import probetools as probe

In [2]:
cohorts = [
    # Ag1000G phase 3 Ghana sample set in Ag3.0
    "AG1000G-GH",
    # Amenta-Etego sample sets in Ag3.3
    #'1190-VO-GH-AMENGA-ETEGO-VMF00013',
    #'1190-VO-GH-AMENGA-ETEGO-VMF00014',
    #'1190-VO-GH-AMENGA-ETEGO-VMF00028',
    #'1190-VO-GH-AMENGA-ETEGO-VMF00029',
    #'1190-VO-GH-AMENGA-ETEGO-VMF00046',
    #'1190-VO-GH-AMENGA-ETEGO-VMF00047',
    # GAARDIAN sample set in Ag3.4
    #'1244-VO-GH-YAWSON-VMF00149',
    'AG1000G-ML-A',
    'AG1000G-BF-A',
    'AG1000G-BF-B',
    'AG1000G-GN-A',
    'AG1000G-GN-B',
    # GAARD Ghana sample set in Ag3.2
    "1244-VO-GH-YAWSON-VMF00051",
    '1245-VO-CI-CONSTANT-VMF00054',
    '1253-VO-TG-DJOGBENOU-VMF00052',
    '1237-VO-BJ-DJOGBENOU-VMF00050'
]

We need:

- VCF file of input data
  - Write to VCF the 28,500,000 to 28,600,000 region for all samples of interest.
- haplotype scaffold
  - Write haps for that region for all samples, format they want
- sample ID file
  - 

In [3]:
contig = '2L'
start = 28_520_000
end = 28_580_000
region = f"{contig}:{start}-{end}"
region

'2L:28520000-28580000'

In [4]:
ag3 = malariagen_data.Ag3(pre=True)

In [5]:
calls = ag3.snp_calls(region=region, sample_sets=cohorts, sample_query="taxon == 'gambiae'")

In [6]:
ac = allel.GenotypeArray(calls['call_genotype']).count_alleles()
seg = ac.is_segregating()

calls = calls.sel(variants=seg)
alleles = calls['variant_allele'].compute()
geno = allel.GenotypeArray(calls['call_genotype'])

In [21]:
pos = allel.SortedIndex(calls['variant_position'])

In [67]:
nonsynons = np.array([])
for i in range(2,9):
    transcript_id = f"AGAP00622{i}-RA"
    
    snp_allele_freqs_df = ag3.snp_allele_frequencies(
        transcript=transcript_id, 
        cohorts="admin1_year", 
        sample_sets=cohorts, 
        drop_invariant=False,
    )
    df = snp_allele_freqs_df.query("max_af > 0.05 & effect == 'NON_SYNONYMOUS_CODING'")
    nonsynons = np.append(nonsynons, df.reset_index().loc[:, 'position'].to_list())


In [68]:
nonsynons

array([28524287., 28524290., 28524291., 28524293., 28524308., 28524309.,
       28524333., 28524341., 28524349., 28524356., 28524356., 28524359.,
       28524360., 28524740., 28524803., 28524812., 28524832., 28524834.,
       28524905., 28524929., 28525051., 28525052., 28525104., 28525153.,
       28525158., 28525238., 28525265., 28525272., 28525348., 28525357.,
       28525469., 28525471., 28525481., 28525519., 28525520., 28525530.,
       28525545., 28525573., 28525576., 28525639., 28525654., 28525654.,
       28525755., 28525757., 28525781., 28525783., 28525786., 28525854.,
       28525857., 28525858., 28525863., 28525867., 28525867., 28525868.,
       28525870., 28525941., 28526034., 28526037., 28526042., 28526046.,
       28526071., 28526089., 28526091., 28526098., 28526102., 28526112.,
       28526115., 28526118., 28526125., 28526657., 28526661., 28526663.,
       28526681., 28526690., 28526703., 28526709., 28526717., 28526724.,
       28526727., 28526826., 28526828., 28526902., 

In [8]:
metadata = ag3.sample_metadata(sample_sets=cohorts, sample_query="taxon == 'gambiae'")

def write_vcf_header(vcf_file, contig):
    """
    Writes a VCF header.
    """
    
    print('##fileformat=VCFv4.1', file=vcf_file)
    # write today's date
    today = date.today().strftime('%Y%m%d')
    print('##fileDate=%s' % today, file=vcf_file)
    # write source
    print('##source=scikit-allel-%s + ZarrToVCF.py' % allel.__version__, file=vcf_file)
    #write refs and contigs 
    print('##reference=resources/reference/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP4.fa', file=vcf_file)
    print('##contig=<ID=2R,length=61545105>', file=vcf_file) if contig == '2R' else None
    print('##contig=<ID=3R,length=53200684>', file=vcf_file) if contig == '3R' else None 
    print('##contig=<ID=2L,length=49364325>', file=vcf_file) if contig == '2L' else None
    print('##contig=<ID=3L,length=41963435>', file=vcf_file) if contig == '3L' else None
    print('##contig=<ID=X,length=24393108>', file=vcf_file) if contig == 'X' else None
    print('##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">', file=vcf_file)

def GenoToPandasToVCF(vcf_file, geno, positions, alleles, contig, nchunks=4):
    
    """
    Converts genotype and POS arrays to vcf, using pd dataframes in chunks. 
    Segregating sites only. Needs REF and ALT arrays.
    """
    refs = alleles[:,0]
    alts = alleles[:,1:]
    refs = refs.astype(str)
    alts = [a +"," + b + "," + c for a,b,c in alts.values.astype(str)]

    probe.log("calculating chunks sizes...")
    chunks = np.round(np.arange(0, geno.shape[0], geno.shape[0]/nchunks)).astype(int).tolist()
    chunks.append(geno.shape[0])

    for idx, chunk in enumerate(chunks[:-1]):

        gn = geno[chunks[idx]:chunks[idx+1]]
        pos = positions[chunks[idx]:chunks[idx+1]]
        ref = refs[chunks[idx]:chunks[idx+1]]
        alt = alts[chunks[idx]:chunks[idx+1]]
        
        # Contruct SNP info DF
        vcf_df = pd.DataFrame({'#CHROM': contig,
                 'POS': pos,
                 'ID': '.',
                 'REF': ref,
                 'ALT': alt,
                 'QUAL': '.',
                 'FILTER': '.',
                 'INFO':'.',
                'FORMAT': 'GT'})

        probe.log(f"Pandas SNP info DataFrame constructed...{idx}")

        # Geno to VCF
        vcf = pd.DataFrame(gn.to_gt().astype(str), columns=metadata['sample_id'])
        probe.log("Concatenating info and genotype dataframes...")
        vcf = pd.concat([vcf_df, vcf], axis=1)

        probe.log(f"Pandas Genotype data constructed...{idx}")

        if (idx==0) is True:
            with open(f"{vcf_file}", 'w') as vcfheader:
                    write_vcf_header(vcfheader, contig)

        probe.log("Writing to .vcf")

        vcf.to_csv(vcf_file, 
                   sep="\t", 
                   index=False,
                   mode='a',
                  header=(idx==0), 
                  line_terminator="\n")

In [72]:
pos_bool = pos.locate_intersection(nonsynons)

In [83]:
#geno = geno.compress(pos_bool[0], axis=0)
alleles = alleles[pos_bool[0]]
pos = pos[pos_bool[0]]

In [87]:
GenoToPandasToVCF("WestAfricaAnGambiae_28_5.vcf", geno, pos, alleles, contig='2L')

calculating chunks sizes...
Pandas SNP info DataFrame constructed...0
Concatenating info and genotype dataframes...
Pandas Genotype data constructed...0
Writing to .vcf
Pandas SNP info DataFrame constructed...1
Concatenating info and genotype dataframes...
Pandas Genotype data constructed...1
Writing to .vcf
Pandas SNP info DataFrame constructed...2
Concatenating info and genotype dataframes...
Pandas Genotype data constructed...2
Writing to .vcf
Pandas SNP info DataFrame constructed...3
Concatenating info and genotype dataframes...
Pandas Genotype data constructed...3
Writing to .vcf


### Scaffolds file

In [135]:
sample_id_order_vcf = metadata['sample_id'].to_numpy()

In [216]:
haps = ag3.haplotypes(region=region, sample_sets=cohorts, analysis='gamb_colu')
sample_id_order_haps = haps['sample_id'].compute().values
bool_ = np.isin(sample_id_order_haps, sample_id_order_vcf)
hap_ac = allel.GenotypeArray(haps['call_genotype']).count_alleles()
hap_seg = hap_ac.is_segregating()
haps = haps.sel(variants=hap_seg,samples=bool_)

In [217]:
alleles = haps['variant_allele'].compute().astype(str)

In [218]:
ids = np.array(["SNP" + a for a in haps['variant_position'].values.astype(str)])

In [219]:
haps_df = pd.DataFrame({'#CHROM': contig,
        'ID': ids,
        'POS': haps['variant_position'].values,
        'REF': alleles[:,0],
        'ALT': alleles[:,1]})
        
haps_geno = allel.GenotypeArray(haps['call_genotype']).to_haplotypes()

haps_geno_df = pd.DataFrame(haps_geno)
allhaps = pd.concat([haps_df, haps_geno_df], axis=1)

In [233]:
pd.Series(haps['sample_id'].values).to_csv("haps.sample", index=False, header=False)

In [232]:
allhaps.to_csv("WestAfricaAnGambiae_28_5.haps", sep="\t", header=False, index=False)

In [226]:
metadata.shape

(1015, 26)

In [225]:
haps

<xarray.Dataset>
Dimensions:           (variants: 11131, alleles: 2, samples: 1015, ploidy: 2)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(11131,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(11131,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(36,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(11131, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(11131, 24, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [227]:
allhaps

,#CHROM,ID,POS,REF,ALT,0,1,2,3,4,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
0,2L,SNP28520002,28520002,C,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2L,SNP28520008,28520008,A,G,1,1,0,1,1,...,0,0,1,0,1,1,1,1,0,0
2,2L,SNP28520016,28520016,A,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2L,SNP28520017,28520017,G,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2L,SNP28520022,28520022,T,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126,2L,SNP28579975,28579975,C,T,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
11127,2L,SNP28579976,28579976,G,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11128,2L,SNP28579986,28579986,C,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11129,2L,SNP28579991,28579991,G,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
